# VQE DIY:
What is VQE?

VQE is an algorithm that find the minimum of the cost function which can be expressed in Pauli matrices.
 
Consider a Hermitian matrix as:
\begin{equation}
\begin{split}
M&= \begin{bmatrix}
-0.2524859
 &0.18121
 \\ 
0.18121
 & -1.8318639
\end{bmatrix}\\&=-1.0421749I+ 0.789689Z+ 0.181210X.
\end{split}
\end{equation}
The cost function is
\begin{equation}
\begin{split}
\langle\psi (\theta)|M|\psi(\theta) \rangle,
\end{split}
\end{equation}
$|𝜓(𝜃)⟩$ is the trial wavefunction.

If we operate $R_y(\theta)$  on  $|0\rangle$ :
\begin{equation}
|\psi(\theta)\rangle=R_y(\theta)|0\rangle=\text{cos}(\theta/2)|0\rangle+\text{sin}(\theta/2)|1\rangle.
\end{equation}
The lowest eigenvalue($M_0$) can be consider as:
\begin{equation}
M_0=\min_\theta\langle0|R_y(\theta)^\dagger MR_y(\theta)|0\rangle=\min_\theta M(\theta)
\end{equation}


In [1]:
import numpy as np 
from qiskit import( QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer)
import math


### Calculating $M(\theta)$ on QC

The expectation value of $Z$ is as
\begin{equation}
\langle Z\rangle=\mathbb{P}_0-\mathbb{P}_1,
\end{equation}
where $(\mathbb{P}_0,\mathbb{P}_1)$ is the probability distribution after the measurement.

If we don't want to measure the state on Z-axis, we can rotate the bloch vector to change the measure axis via the single-qubit gate.

The expectation value of $X$ is as
\begin{equation}
\langle R_y(\pi/2)ZR_y(-\pi/2)\rangle=\mathbb{P}_0-\mathbb{P}_1.
\end{equation}


In [17]:
shot=10000
simulator = Aer.get_backend('qasm_simulator')

def M(x0):
    theta=x0[0]
    M=-1.0421749
    ####caculate the expectation value of 0.789689𝑍
    qr = QuantumRegister(1) # initiate the circuit
    cr = ClassicalRegister(1) #
    Cix= QuantumCircuit(qr, cr) #
    #Cix.x(0) initial in |1>
    Cix.ry(theta,0) #the ansatz 
    Cix.measure(0,0)       
    job = execute(Cix, simulator, shots=shot) 
    result = job.result().get_counts()#collect the data
    
    P_0=result.get('0',0)/shot
    P_1=result.get('1',0)/shot
    temp=P_0-P_1  
    #temp is the expectation value of Z
    M+=temp*0.789689
    
    ####caculate the expectation value of 0.181210𝑋
    qr = QuantumRegister(1)
    cr = ClassicalRegister(1)
    Cix= QuantumCircuit(qr, cr)
    #Cix.x(0)  initial in |1>
    Cix.ry(theta,0)# the ansatz
    Cix.ry(-math.pi/2,0)# rotate the measurement frame
    Cix.measure(0,0)       
    job = execute(Cix, simulator, shots=shot) 
    result = job.result().get_counts()
        
    P_0=result.get('0',0)/shot
    P_1=result.get('1',0)/shot
    temp=P_0-P_1 
    #temp is the expectation value of X    
    M+=temp*0.181210
    return M

 ### Using the classical optimization algorithm
 scipy.optimize.minimize:
 https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize


In [18]:
from scipy.optimize import minimize
sol = minimize(M,[0],method='COBYLA',options={'rhobeg': 1, 'maxiter': 100, 'disp': False, 'catol': 0.0001})
## classical optimization algorithm‘Nelder-Mead’,‘Powell’,‘L-BFGS-B’,‘COBYLA’ ,‘SLSQP’.........

### The solution of VQE

In [19]:
print(sol.fun,sol.nfev)

-1.8537285926000002 21


In [20]:
import scipy.linalg as la
H= np.array([[-0.2524859,0.18121],[0.18121,-1.8318639]])
exacteigenvalue = la.eig(H)
exacteigenvalue[0][1]

(-1.8523883168359594+0j)

 ### Error rate

In [21]:
error=(sol.fun-exacteigenvalue[0][1])/exacteigenvalue[0][1]
error

(0.0007235393096897416-0j)